## 다우존스의 ETF 중 하나인 DIA와 코스피의 ETF인 코덱스를 Catboost로 예측
- 기준 x, o의 차이 : 기준 x는 변동율이 0 이상, 이하로 나뉨, 기준 o는 0.003을 기준으로 잡아 1, 0, -1로 분류
0.003 기준 : Stock Market Trend Forecasting Based on Multiple Textual Features: A Deep Learning Method

- 수정 전과 후의 차이 : 기존 참고했던 논문의 핵심 포인트가 다우존스 시장이 개장하기 전에 종가가 나온 주가들로 예측해서 동일한 날짜로 예측을 했음. 하지만 금, 은, 유가, 환율은 개장이 끝나도 종가가 나오지 않음. 그래서 논문을 그대로 따라한 것이 수정 전 결과이고, 내 의구심으로 코드를 수정한 결과가 수정 후 결과이다.

In [16]:
import yfinance as yf
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import itertools
from itertools import combinations

from functools import reduce

In [ ]:
# 한번에 뽑는 코드
stock = ['^DJI','^N225', "^HSI", "^FTSE", "^GDAXI","^AXJO","^HSI", "EURUSD=X", "AUDUSD=X", "JPYUSD=X", "USD","GC=F","SI=F", "PL=F",'CL=F']
target = '^DJI'
start_date = "2011-12-30" # 2012-01-01 이후 첫번째 데이터가 Nan인 경우 선형 보간법을 사용하기 위해 앞의 날짜부터 가져옴
end_date = "2022-04-30"

stock_df = yf.download(stock,start = start_date, end = end_date).Close
stock_df = stock_df.reset_index()
stock_df = stock_df.dropna(subset = [target])
stock_df[stock_df.columns.difference(['Date'])] = stock_df[stock_df.columns.difference(['Date'])].interpolate(method='linear',axis=0)
stock_df = stock_df.dropna()

difference = 1
stock_df = stock_df[stock_df.columns.difference(["Date"])]
stock_df = stock_df.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
stock_df = stock_df.dropna()

#stock_df = stock_df.apply(lambda y: y.apply(lambda x: 1 if x > 0 else (-1 if x<0 else 0)))
stock_df[stock_df.columns.difference(['^DJI','^N225', "^HSI", "^FTSE", "^GDAXI","^AXJO","^HSI",])] = stock_df[stock_df.columns.difference(['^DJI','^N225', "^HSI", "^FTSE", "^GDAXI","^AXJO","^HSI",])].shift(difference)
stock_df["Close"] = stock_df["Close"].apply(lambda y: 1 if y >= 0.003 else (-1 if y<=-0.003 else 0))

stock_df = stock_df.apply(lambda x: x/abs(x), axis = 0)
stock_df = stock_df.dropna()
stock_df = stock_df.reset_index(drop=True)

stock_df[stock_df.columns.difference([target])] = stock_df[stock_df.columns.difference([target])].shift(difference)
stock_df = stock_df.dropna()

x = stock_df[stock_df.columns.difference([target])]
y = stock_df[[target]]

x.columns = stock_df.columns.difference([target])
y.columns = [target]

x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

model = CatBoostClassifier()
model.fit(x.train, y.train)
preds_class = model.predict(x.test)

[*********************100%***********************]  14 of 14 completed
Learning rate set to 0.013102
0:	learn: 0.6925307	total: 2.25ms	remaining: 2.24s
1:	learn: 0.6918547	total: 4.19ms	remaining: 2.09s
2:	learn: 0.6912835	total: 6.18ms	remaining: 2.05s
3:	learn: 0.6905092	total: 8.05ms	remaining: 2s
4:	learn: 0.6897849	total: 9.8ms	remaining: 1.95s
5:	learn: 0.6890697	total: 11.5ms	remaining: 1.9s
6:	learn: 0.6884273	total: 13.1ms	remaining: 1.86s
7:	learn: 0.6878342	total: 14.9ms	remaining: 1.84s
8:	learn: 0.6877370	total: 15.7ms	remaining: 1.73s
9:	learn: 0.6872132	total: 17.4ms	remaining: 1.72s
10:	learn: 0.6865601	total: 19.2ms	remaining: 1.72s
11:	learn: 0.6859704	total: 21.9ms	remaining: 1.8s
12:	learn: 0.6853517	total: 24.5ms	remaining: 1.86s
13:	learn: 0.6852657	total: 25.6ms	remaining: 1.8s
14:	learn: 0.6846507	total: 28.3ms	remaining: 1.85s
15:	learn: 0.6840596	total: 30.1ms	remaining: 1.85s
16:	learn: 0.6834063	total: 32ms	remaining: 1.85s
17:	learn: 0.6826817	total: 33.8ms

/tmp/ipykernel_3402774/3101122023.py:34: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_3402774/3101122023.py:35: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)


68:	learn: 0.6600933	total: 93.5ms	remaining: 1.26s
69:	learn: 0.6595905	total: 94.7ms	remaining: 1.26s
70:	learn: 0.6590863	total: 95.8ms	remaining: 1.25s
71:	learn: 0.6585845	total: 96.9ms	remaining: 1.25s
72:	learn: 0.6585644	total: 97.4ms	remaining: 1.24s
73:	learn: 0.6582534	total: 98.5ms	remaining: 1.23s
74:	learn: 0.6578878	total: 99.6ms	remaining: 1.23s
75:	learn: 0.6573796	total: 101ms	remaining: 1.22s
76:	learn: 0.6569135	total: 102ms	remaining: 1.22s
77:	learn: 0.6564726	total: 103ms	remaining: 1.22s
78:	learn: 0.6561416	total: 104ms	remaining: 1.21s
79:	learn: 0.6557870	total: 105ms	remaining: 1.21s
80:	learn: 0.6554625	total: 106ms	remaining: 1.2s
81:	learn: 0.6554322	total: 107ms	remaining: 1.19s
82:	learn: 0.6550408	total: 108ms	remaining: 1.19s
83:	learn: 0.6546047	total: 109ms	remaining: 1.19s
84:	learn: 0.6545109	total: 110ms	remaining: 1.18s
85:	learn: 0.6540402	total: 111ms	remaining: 1.18s
86:	learn: 0.6536532	total: 112ms	remaining: 1.17s
87:	learn: 0.6533150	tota

In [ ]:
sum(y.test[target].values == preds_class.reshape(len(x.test)))/len(y.test)

0.5046480743691899

In [127]:
## 따로 뽑아서 붙이는 코드
start_date = "2011-12-30"
end_date = "2022-05-04"

kodex = yf.download('069500.KS',start = start_date, end = end_date)
kodex = kodex.reset_index()[["Date","Close"]]
kodex = kodex.dropna(axis=0)
kodex.columns = ["Kodex_Date","Kodex"]

dowJ = yf.download('^DJI',start = start_date, end = end_date)
dowJ = dowJ.reset_index()[["Date","Close"]]
dowJ = dowJ.dropna(axis=0)
dowJ.columns = ["DowJ_Date","DowJ"]

kospi = yf.download('^KS11',start = start_date, end = end_date)
kospi = kospi.reset_index()[["Date","Close"]]
kospi = kospi.dropna(axis=0)
kospi.columns = ["Kospi_Date","Kospi"]

kospi200 = yf.download('^KS200',start = start_date, end = end_date)
kospi200 = kospi200.reset_index()[["Date","Close"]]
kospi200 = kospi200.dropna(axis=0)
kospi200.columns = ["kospi200_Date","kospi200"]

kosdaq = yf.download('^KQ11',start = start_date, end = end_date)
kosdaq = kosdaq.reset_index()[["Date","Close"]]
kosdaq = kosdaq.dropna(axis=0)
kosdaq.columns = ["Kosdaq_Date","Kosdaq"]

samsung = yf.download('005930.KS',start = start_date, end = end_date)
samsung = samsung.reset_index()[["Date","Close"]]
samsung = samsung.dropna(axis=0)
samsung.columns = ["SamS_Date","SamS"]

krw = yf.download('KRW=X',start = start_date, end = end_date)
krw = krw.reset_index()[["Date","Close"]]
krw = krw.dropna(axis=0)
krw.columns = ["KRW_Date","KRW"]

gold = yf.download('GC=F',start = start_date, end = end_date)
gold = gold.reset_index()[["Date","Close"]]
gold = gold.dropna(axis=0)
gold.columns = ["Gold_Date","Gold"]

silver = yf.download('SI=F',start = start_date, end = end_date)
silver = silver.reset_index()[["Date","Close"]]
silver = silver.dropna(axis=0)
silver.columns = ["Silver_Date","Silver"]

platinum = yf.download('PL=F',start = start_date, end = end_date)
platinum = platinum.reset_index()[["Date","Close"]]
platinum = platinum.dropna(axis=0)
platinum.columns = ["Plati_Date","Platinum"]

oil = yf.download('CL=F',start = start_date, end = end_date)
oil = oil.reset_index()[["Date","Close"]]
oil = oil.dropna(axis=0)
oil.columns = ["Oil_Date","Oil"]

nasdaq = yf.download('^IXIC',start = start_date, end = end_date)
nasdaq = nasdaq.reset_index()[["Date","Close"]]
nasdaq = nasdaq.dropna(axis=0)
nasdaq.columns = ["NasD_Date","NasD"]

ftse = yf.download('^FTSE',start = start_date, end = end_date)
ftse = ftse.reset_index()[["Date","Close"]]
ftse = ftse.dropna(axis=0)
ftse.columns = ["FTSE_Date","FTSE"]

sp500 = yf.download('^GSPC',start = start_date, end = end_date)
sp500 = sp500.reset_index()[["Date","Close"]]
sp500 = sp500.dropna(axis=0)
sp500.columns = ["SP500_Date","SP500"]

hang_seng = yf.download('^HSI',start = start_date, end = end_date)
hang_seng = hang_seng.reset_index()[["Date","Close"]]
hang_seng = hang_seng.dropna(axis=0)
hang_seng.columns = ["HS_Date","HangS"]

stock_lst = [kodex, dowJ, kospi, kosdaq, samsung, krw, gold, silver, platinum, oil, nasdaq, ftse, sp500, hang_seng, kospi200]
stock_vari = []
for i in range(len(stock)+1):
    for vari in combinations(stock, i):
        stock_vari.append(list(vari))

stock_vari = [['Kodex_Date', 'Kodex']+stock_vari[i] for i in range(len(stock_vari))]

i = 0
pred = []
for stock_name in stock_vari[1:]:
    i += 1
    if i % 100 == 0:
        print("---------------- {} 번째 완료 --------------".format(i))
    stock_df = reduce(lambda x, y: pd.merge(x, y, left_on = x.columns[0], right_on = y.columns[0], how = "left"), stock_lst)
    stock_df = stock_df[stock_name]

    # 선형보간법
    stock_df[stock_df.columns.difference(['Kodex_Date'])] = stock_df[stock_df.columns.difference(['Kodex_Date'])].interpolate(method='linear',axis=0)
    stock_df = stock_df.drop(["Kodex_Date"], axis = 1)

    difference = 1
    stock_df = stock_df.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    stock_df[stock_df.columns.difference(['Kodex'])] = stock_df[stock_df.columns.difference(['Kodex'])].shift(difference) 
    stock_df[stock_df.columns.difference(['Kodex', 'Kospi', 'Kosdaq', 'SamS', 'KRW', "HangS"])] = stock_df[stock_df.columns.difference(['Kodex', 'Kospi', 'Kosdaq', 'SamS', 'KRW', "HangS"])].shift(difference)
    #stock_df = stock_df.apply(lambda x: x/abs(x), axis = 0)
    stock_df = stock_df.apply(lambda y: y.apply(lambda x: 1 if x >= 0.003 else (-1 if x <= -0.003 else 0)))
    stock_df = stock_df.iloc[2:len(stock_df)]
    stock_df = stock_df.fillna(0)

    train_ind = int(len(stock_df)*0.7)
    train = stock_df.iloc[0:train_ind]
    test = stock_df.iloc[train_ind:len(stock_df)]

    x_n = stock_df.columns.drop(["Kodex"])
    y_n = "Kodex"

    model = CatBoostClassifier(has_time = True, verbose = 0)
    model.fit(train[x_n], train[[y_n]])
    preds_class = model.predict(test[x_n])
    p = sum(test[[y_n]].values == preds_class)/len(preds_class)
    pred.append(p)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [126]:
# 기준 x
max(pred),stock_vari[1+pred.index(max(pred))]

(array([0.56973684]),
 ['Kodex_Date', 'Kodex', 'SamS', 'KRW', 'Gold', 'Silver', 'Platinum', 'SP500'])

In [ ]:
#                 수정 전         수정 후
# DowJ -> 기준 x   0.7       ->    0.62
#         기준 o   0.55      ->    0.46
# DIA -> 기준 x    0.7       ->    0.6
#        기준 o    0.53      ->    0.44
# KODEX -> 기준 x  0.57
#       -> 기준 o  0.39